In [1]:
#*******************************************************************************************
 #
 #  File Name:  etl_mini_project.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, etl_mini_project.ipynb, builds on part of an
 #      ETL pipeline using Python, Pandas, and Python regular expressions. The script 
 #      extracts the data from two Excel files, crowdfunding.xlsx and contacts.xlsx, 
 #      and stores it in Pandas DataFrames before transforming and writing the results
 #      to four CSV files.
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  09/21/2023      Initial Development                     Nicholas J. George
 #
 #******************************************************************************************/

import log_subroutines
import pandas_process_functions

import numpy as np
import pandas as pd

from datetime import datetime as dt

In [2]:
CONSTANT_LOCAL_FILE_NAME = 'etl_mini_project.ipynb'


CONSTANT_CROWDFUNDING_FILE_PATH = './resources/crowdfunding.xlsx'

CONSTANT_CATEGORY_FILE_PATH = './resources/category.csv'

CONSTANT_SUBCATEGORY_FILE_PATH = './resources/subcategory.csv'

CONSTANT_CAMPAIGN_FILE_PATH = './resources/campaign.csv'

CONSTANT_CONTACTS_EXCEL_FILE_PATH = './resources/contacts.xlsx'

CONSTANT_CONTACTS_CSV_FILE_PATH = './resources/contacts.csv'


log_subroutines.set_log_mode(False)

log_subroutines.set_image_mode(False)

log_subroutines.begin_program('etl_mini_project')


pd.set_option('max_colwidth', 400)

# <br> **1. Category and Subcategory DataFrames**

## **1.1 Crowdfunding DataFrame**

### Extract Crowdfunding DataFrame from Excel Workbook

In [3]:
excel_crowdfunding_dtype_dictionary \
    = {'cf_id': int, 
       'contact_id': int,
       'company_name': str,
       'blurb': str,
       'goal': float,
       'pledged': float,
       'outcome': str,
       'backers_count': int,
       'country': str,
       'currency': str,
       'staff_pick': bool,
       'spotlight': bool,
       'category & sub-category': str,
       'Value': float}

excel_crowdfunding_converter_dictionary \
    = {'launched_at': dt.fromtimestamp,
       'deadline': dt.fromtimestamp}

crowdfunding_dataframe \
    = pd.read_excel \
        (open(CONSTANT_CROWDFUNDING_FILE_PATH, 'rb'),
         sheet_name = 'crowdfunding_info',
         header = 0,
         dtype = excel_crowdfunding_dtype_dictionary,
         converters = excel_crowdfunding_converter_dictionary) 

log_subroutines.log_write_object(crowdfunding_dataframe)

### Display Initial Crowdfunding DataFrame

In [4]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_dataframe,
         'Table 1.1.1: Initial Crowdfunding DataFrame')

cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
147,"4,661","Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food/food trucks
"1,621","3,765",Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music/rock
"1,812","4,187","Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology/web
"2,156","4,941","Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music/rock
"1,365","2,199",Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater/plays
"2,057","5,650",Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater/plays
"1,894","5,889","Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video/documentary
"2,669","4,842",Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater/plays
"1,114","3,280",Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater/plays
970,"5,468","Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music/electric music


### Crowdfunding DataFrame Information

In [5]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'Table 1.1.2: Crowdfunding DataFrame Information:\n' 
         + '\033[0m')

crowdfunding_dataframe \
    .info \
        (verbose = True,
         memory_usage = True, 
         show_counts = True)

Table 1.1.2: Crowdfunding DataFrame Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   cf_id                    1000 non-null   int64         
 1   contact_id               1000 non-null   int64         
 2   company_name             1000 non-null   object        
 3   blurb                    1000 non-null   object        
 4   goal                     1000 non-null   float64       
 5   pledged                  1000 non-null   float64       
 6   outcome                  1000 non-null   object        
 7   backers_count            1000 non-null   int64         
 8   country                  1000 non-null   object        
 9   currency                 1000 non-null   object        
 10  launched_at              1000 non-null   datetime64[ns]
 11  deadline                 1000 non-null   datet

### Crowdfunding DataFrame Description

In [6]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_dataframe,
         'Table 1.1.3: Crowdfunding DataFrame Description') \
    .hide \
        (['cf_id', 'contact_id'],
         axis = 1)

company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food/food trucks
Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music/rock
"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology/web
"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music/rock
Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater/plays
Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater/plays
"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video/documentary
Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater/plays
Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater/plays
"Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music/electric music


### Crowdfunding DataFrame Memory Usage

In [7]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_dataframe,
         'Table 1.1.4: Crowdfunding DataFrame Memory Usage')

cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
147,"4,661","Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food/food trucks
"1,621","3,765",Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music/rock
"1,812","4,187","Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology/web
"2,156","4,941","Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music/rock
"1,365","2,199",Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater/plays
"2,057","5,650",Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater/plays
"1,894","5,889","Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video/documentary
"2,669","4,842",Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater/plays
"1,114","3,280",Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater/plays
970,"5,468","Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music/electric music


## **1.2 Extract Categories and SubCategories From One Column**

### Crowdfunding DataFrame's Column Title List

In [8]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + "This is a List of the Crowdfunding DataFrame's column titles:\n\n" \
         + '\033[0m' \
         + str(crowdfunding_dataframe.keys()))

This is a List of the Crowdfunding DataFrame's column titles:

Index(['cf_id', 'contact_id', 'company_name', 'blurb', 'goal', 'pledged',
       'outcome', 'backers_count', 'country', 'currency', 'launched_at',
       'deadline', 'staff_pick', 'spotlight', 'category & sub-category'],
      dtype='object')


### Category and Subcategory Separation

In [9]:
if 'category & sub-category' in crowdfunding_dataframe.columns:

    crowdfunding_dataframe[['category', 'subcategory']] \
        = crowdfunding_dataframe['category & sub-category'] \
            .str \
            .split('/', expand = True)

    crowdfunding_dataframe \
        .drop \
            ('category & sub-category', 
             axis = 1, 
             inplace = True)

    log_subroutines.log_write_object(crowdfunding_dataframe)

### Display Updated Crowdfunding DataFrame

In [10]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_dataframe,
         'Table 1.2.1: Updated Crowdfunding DataFrame') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category,subcategory
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food,food trucks
1621,3765,Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music,rock
1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology,web
2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music,rock
1365,2199,Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater,plays
2057,5650,Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater,plays
1894,5889,"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video,documentary
2669,4842,Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater,plays
1114,3280,Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater,plays
970,5468,"Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music,electric music


### Category List

In [11]:
categories_string_list \
    = sorted(crowdfunding_dataframe['category'].unique().tolist())

log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'This is the list of {:,} categories from the Crowdfunding DataFrame ' \
                .format(len(categories_string_list)) \
         + 'in alphabetical order:\n\n' \
         + '\033[0m' \
         + str(categories_string_list))

This is the list of 9 categories from the Crowdfunding DataFrame in alphabetical order:

['film & video', 'food', 'games', 'journalism', 'music', 'photography', 'publishing', 'technology', 'theater']


### Sub-Category List

In [12]:
subcategories_string_list \
    = sorted(crowdfunding_dataframe['subcategory'].unique().tolist())

log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'This is the list of {:,} subcategories from the Crowdfunding DataFrame '
                .format(len(subcategories_string_list)) \
         + 'in alphabetical order:\n\n' \
         + '\033[0m' \
         + str(subcategories_string_list))

This is the list of 24 subcategories from the Crowdfunding DataFrame in alphabetical order:

['animation', 'audio', 'documentary', 'drama', 'electric music', 'fiction', 'food trucks', 'indie rock', 'jazz', 'metal', 'mobile games', 'nonfiction', 'photography books', 'plays', 'radio & podcasts', 'rock', 'science fiction', 'shorts', 'television', 'translations', 'video games', 'wearables', 'web', 'world music']


### Category and Subcategory IDs

In [13]:
category_array_limit_integer = len(categories_string_list) + 1

subcategory_array_limit_integer = len(subcategories_string_list) + 1


category_ids_numpy_array = np.arange(1, category_array_limit_integer)

subcategory_ids_numpy_array = np.arange(1, subcategory_array_limit_integer)


log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'This is the numPy Array for Category IDs:\n'
         + '\033[0m' \
         + str(category_ids_numpy_array) \
         + '\n')

log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'This is the numPy Array for Subcategory IDs:\n'
         + '\033[0m' \
         + str(subcategory_ids_numpy_array))

This is the numPy Array for Category IDs:
[1 2 3 4 5 6 7 8 9]

This is the numPy Array for Subcategory IDs:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


## **1.3 Create and Transform Category and SubCategory DataFrames**

### Create Category DataFrame

In [14]:
crowdfunding_category_dataframe \
    = pd.DataFrame \
        ({'category_id': category_ids_numpy_array,
          'category': categories_string_list})

log_subroutines.log_write_object(crowdfunding_category_dataframe)

### Update Category ID with Prefix

In [15]:
crowdfunding_category_dataframe['category_id'] \
    = 'cat' + crowdfunding_category_dataframe['category_id'].astype(str)

log_subroutines.log_write_object(crowdfunding_category_dataframe['category_id'])

### Display Category DataFrame

In [16]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_category_dataframe,
         'Table 1.3.1: Category DataFrame')

category_id,category
cat1,film & video
cat2,food
cat3,games
cat4,journalism
cat5,music
cat6,photography
cat7,publishing
cat8,technology
cat9,theater


### Create Subcategory DataFrame

In [17]:
crowdfunding_subcategory_dataframe \
    = pd.DataFrame \
        ({'subcategory_id': subcategory_ids_numpy_array,
          'subcategory': subcategories_string_list})

### Update Subcategory ID with Prefix

In [18]:
crowdfunding_subcategory_dataframe['subcategory_id'] \
    = 'subcat' + crowdfunding_subcategory_dataframe['subcategory_id'].astype(str)

log_subroutines.log_write_object(crowdfunding_subcategory_dataframe['subcategory_id'])

### Display Subcategory DataFrame

In [19]:
pandas_process_functions \
    .return_formatted_table \
        (crowdfunding_subcategory_dataframe,
         'Table 1.3.2: Subcategory DataFrame')

subcategory_id,subcategory
subcat1,animation
subcat2,audio
subcat3,documentary
subcat4,drama
subcat5,electric music
subcat6,fiction
subcat7,food trucks
subcat8,indie rock
subcat9,jazz
subcat10,metal


## **1.4 Export Category and Subcategory DataFrames to CSV Files**

In [20]:
crowdfunding_category_dataframe.to_csv(CONSTANT_CATEGORY_FILE_PATH, index = False)

crowdfunding_subcategory_dataframe.to_csv(CONSTANT_SUBCATEGORY_FILE_PATH, index = False)

# <br> **2. Campaign DataFrame**

## **2.1 Create Campaign DataFrame**

### Initial Campaign DataFrame from Crowdfunding DataFrame

In [21]:
campaign_dataframe = crowdfunding_dataframe.copy()

log_subroutines.log_write_object(campaign_dataframe)

### Display Initial Campaign DataFrame

In [22]:
pandas_process_functions \
    .return_formatted_table \
        (campaign_dataframe,
         'Table 2.1.1: Initial Campaign DataFrame') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category,subcategory
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food,food trucks
1621,3765,Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music,rock
1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology,web
2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music,rock
1365,2199,Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater,plays
2057,5650,Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater,plays
1894,5889,"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video,documentary
2669,4842,Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater,plays
1114,3280,Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater,plays
970,5468,"Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music,electric music


## **2.2 Transform Campaign DataFrame**

### Rename Appropriate Campaign DataFrame Columns

In [23]:
campaign_dataframe \
    .rename \
        (columns \
            = {'blurb': 'description',
               'launched_at': 'launch_date',
               'deadline': 'end_date'},
         inplace = True)

log_subroutines.log_write_object(campaign_dataframe)

### Display Campaign DataFrame With Renamed Columns

In [24]:
pandas_process_functions \
    .return_formatted_table \
        (campaign_dataframe,
         'Table 2.2.1: Campaign DataFrame With Renamed Columns') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,staff_pick,spotlight,category,subcategory
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food,food trucks
1621,3765,Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music,rock
1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology,web
2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music,rock
1365,2199,Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater,plays
2057,5650,Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater,plays
1894,5889,"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video,documentary
2669,4842,Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater,plays
1114,3280,Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater,plays
970,5468,"Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music,electric music


### Convert Appropriate DataFrame Columns to `float` Data Type

### The script reads the 'goal' and 'pledged' values as float data types upon extraction from the MS Excel Workbook, crowdfunding.xlsx.  The extraction process uses the data types defined in the ETLMiniProjectConstants.py file's excelCrowdfundingDTypeDictionary.

### Display DataFrame Column Data Types

In [25]:
pandas_process_functions \
    .return_formatted_table \
        (campaign_dataframe,
         'Table 2.2.2: Campaign DataFrame Column Data Types')

cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,staff_pick,spotlight,category,subcategory
147,"4,661","Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12 23:00:00,2021-02-28 23:00:00,False,False,food,food trucks
"1,621","3,765",Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24 23:00:00,2021-05-24 22:00:00,False,True,music,rock
"1,812","4,187","Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16 23:00:00,2021-12-29 23:00:00,False,False,technology,web
"2,156","4,941","Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20 22:00:00,2022-01-16 23:00:00,False,False,music,rock
"1,365","2,199",Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20 23:00:00,2021-08-22 22:00:00,False,False,theater,plays
"2,057","5,650",Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10 23:00:00,2021-08-28 22:00:00,False,False,theater,plays
"1,894","5,889","Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30 22:00:00,2021-05-10 22:00:00,False,False,film & video,documentary
"2,669","4,842",Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21 23:00:00,2021-09-20 22:00:00,False,False,theater,plays
"1,114","3,280",Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07 22:00:00,2021-03-09 23:00:00,False,False,theater,plays
970,"5,468","Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12 22:00:00,2021-08-30 22:00:00,False,False,music,electric music


### Convert Appropriate DataFrame Column Values from Datetime to Dates

In [26]:
campaign_dataframe['launch_date'] \
    = pd.to_datetime(campaign_dataframe['launch_date']).dt.date

log_subroutines.log_write_object(campaign_dataframe['launch_date'])

In [27]:
campaign_dataframe['end_date'] \
    = pd.to_datetime(campaign_dataframe['end_date']).dt.date

log_subroutines.log_write_object(campaign_dataframe['end_date'])

### Display Campaign DataFrame With Dates

In [28]:
pandas_process_functions \
    .return_formatted_table \
        (campaign_dataframe,
         'Table 2.2.3: Campaign DataFrame With Dates') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,staff_pick,spotlight,category,subcategory
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12,2021-02-28,False,False,food,food trucks
1621,3765,Odom Inc,Managed bottom-line architecture,"1,400.00","14,560.00",successful,158,US,USD,2021-01-24,2021-05-24,False,True,music,rock
1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,"108,400.00","142,523.00",successful,"1,425",AU,AUD,2020-12-16,2021-12-29,False,False,technology,web
2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,"4,200.00","2,477.00",failed,24,US,USD,2021-10-20,2022-01-16,False,False,music,rock
1365,2199,Larson-Little,Proactive foreground core,"7,600.00","5,265.00",failed,53,US,USD,2020-12-20,2021-08-22,False,False,theater,plays
2057,5650,Harris Group,Open-source optimizing database,"7,600.00","13,195.00",successful,174,DK,DKK,2020-12-10,2021-08-28,False,False,theater,plays
1894,5889,"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,"5,200.00","1,090.00",failed,18,GB,GBP,2020-07-30,2021-05-10,False,False,film & video,documentary
2669,4842,Carter-Guzman,Centralized cohesive challenge,"4,500.00","14,741.00",successful,227,DK,DKK,2020-12-21,2021-09-20,False,False,theater,plays
1114,3280,Nunez-Richards,Exclusive attitude-oriented intranet,"110,100.00","21,946.00",live,708,DK,DKK,2020-04-07,2021-03-09,False,False,theater,plays
970,5468,"Rangel, Holt and Jones",Open-source fresh-thinking model,"6,200.00","3,208.00",failed,44,US,USD,2021-08-12,2021-08-30,False,False,music,electric music


## **2.3 Merge and Clean Campaign DataFrame**

### Merge Campaign, Category, and Subcategory DataFrames

In [29]:
campaign_category_dataframe \
    = campaign_dataframe \
        .merge \
            (crowdfunding_category_dataframe, 
             on = 'category', 
             how = 'left')

log_subroutines.log_write_object(campaign_dataframe['end_date'])

In [30]:
merged_campaign_dataframe \
    = pd \
        .merge \
            (campaign_category_dataframe, 
             crowdfunding_subcategory_dataframe,
             on = 'subcategory')

log_subroutines.log_write_object(merged_campaign_dataframe)

### Display Merged Campaign DataFrame

In [31]:
pandas_process_functions \
    .return_formatted_table \
        (merged_campaign_dataframe,
         'Table 2.3.1: Merged Campaign DataFrame') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,staff_pick,spotlight,category,subcategory,category_id,subcategory_id
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12,2021-02-28,False,False,food,food trucks,cat2,subcat7
1175,2288,Werner-Bryant,Virtual uniform frame,"1,800.00","7,991.00",successful,222,US,USD,2020-06-19,2021-01-29,False,False,food,food trucks,cat2,subcat7
873,2067,Stewart LLC,Cloned bi-directional architecture,"1,300.00","12,047.00",successful,113,US,USD,2020-11-28,2021-06-10,False,False,food,food trucks,cat2,subcat7
2568,5989,Castillo-Carey,Cross-platform solution-oriented process improvement,"142,400.00","21,307.00",failed,296,US,USD,2020-05-04,2021-05-29,False,False,food,food trucks,cat2,subcat7
1211,3307,"Wright, Hartman and Yu",User-friendly tertiary array,"3,300.00","12,437.00",successful,131,US,USD,2021-01-28,2021-02-12,False,False,food,food trucks,cat2,subcat7
990,3489,Morgan-Martinez,Mandatory tertiary implementation,"148,500.00","4,756.00",canceled,55,AU,AUD,2021-03-22,2021-12-05,False,False,food,food trucks,cat2,subcat7
2228,4404,Alexander-Williams,Triple-buffered cohesive structure,"7,200.00","6,927.00",failed,210,US,USD,2021-04-16,2021-07-07,False,False,food,food trucks,cat2,subcat7
509,1867,Mcknight-Freeman,Upgradable scalable methodology,"8,300.00","6,543.00",canceled,82,US,USD,2020-08-12,2021-04-21,False,False,food,food trucks,cat2,subcat7
966,4840,Huff LLC,Face-to-face clear-thinking Local Area Network,"121,500.00","119,830.00",failed,"2,179",US,USD,2021-04-09,2021-09-30,True,False,food,food trucks,cat2,subcat7
2765,5565,Ho-Harris,Versatile cohesive encoding,"7,300.00",717.00,failed,10,US,USD,2020-11-30,2021-09-27,False,False,food,food trucks,cat2,subcat7


### Clean DataFrame by Dropping Unwanted Columns

In [32]:
clean_merged_campaign_dataframe \
    = merged_campaign_dataframe \
        .drop \
            (['staff_pick', 
              'spotlight', 
              'category', 
              'subcategory'], 
             axis = 1)

log_subroutines.log_write_object(clean_merged_campaign_dataframe)

### Display Clean Merged Campaign DataFrame

In [33]:
pandas_process_functions \
    .return_formatted_table \
        (clean_merged_campaign_dataframe,
         'Table 2.3.2: Clean Merged Campaign DataFrame (Final)') \
    .format \
        ({'cf_id': '{:}',
          'contact_id': '{:}',
          'goal': '{:,.2f}',
          'pledged': '{:,.2f}',
          'backers_count': '{:,}'})

cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,category_id,subcategory_id
147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.00,0.00,failed,0,CA,CAD,2020-02-12,2021-02-28,cat2,subcat7
1175,2288,Werner-Bryant,Virtual uniform frame,"1,800.00","7,991.00",successful,222,US,USD,2020-06-19,2021-01-29,cat2,subcat7
873,2067,Stewart LLC,Cloned bi-directional architecture,"1,300.00","12,047.00",successful,113,US,USD,2020-11-28,2021-06-10,cat2,subcat7
2568,5989,Castillo-Carey,Cross-platform solution-oriented process improvement,"142,400.00","21,307.00",failed,296,US,USD,2020-05-04,2021-05-29,cat2,subcat7
1211,3307,"Wright, Hartman and Yu",User-friendly tertiary array,"3,300.00","12,437.00",successful,131,US,USD,2021-01-28,2021-02-12,cat2,subcat7
990,3489,Morgan-Martinez,Mandatory tertiary implementation,"148,500.00","4,756.00",canceled,55,AU,AUD,2021-03-22,2021-12-05,cat2,subcat7
2228,4404,Alexander-Williams,Triple-buffered cohesive structure,"7,200.00","6,927.00",failed,210,US,USD,2021-04-16,2021-07-07,cat2,subcat7
509,1867,Mcknight-Freeman,Upgradable scalable methodology,"8,300.00","6,543.00",canceled,82,US,USD,2020-08-12,2021-04-21,cat2,subcat7
966,4840,Huff LLC,Face-to-face clear-thinking Local Area Network,"121,500.00","119,830.00",failed,"2,179",US,USD,2021-04-09,2021-09-30,cat2,subcat7
2765,5565,Ho-Harris,Versatile cohesive encoding,"7,300.00",717.00,failed,10,US,USD,2020-11-30,2021-09-27,cat2,subcat7


### Clean Merged Campaign DataFrame Information

In [34]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'Table 2.3.3: Clean Merged Campaign DataFrame Information:\n' 
         + '\033[0m')

clean_merged_campaign_dataframe \
    .info \
        (verbose = True,
         memory_usage = True, 
         show_counts = True)

Table 2.3.3: Clean Merged Campaign DataFrame Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cf_id           1000 non-null   int64  
 1   contact_id      1000 non-null   int64  
 2   company_name    1000 non-null   object 
 3   description     1000 non-null   object 
 4   goal            1000 non-null   float64
 5   pledged         1000 non-null   float64
 6   outcome         1000 non-null   object 
 7   backers_count   1000 non-null   int64  
 8   country         1000 non-null   object 
 9   currency        1000 non-null   object 
 10  launch_date     1000 non-null   object 
 11  end_date        1000 non-null   object 
 12  category_id     1000 non-null   object 
 13  subcategory_id  1000 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 109.5+ KB


### Clean Merged Campaign DataFrame Description

In [35]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'Table 2.3.4: Clean Merged Campaign DataFrame Description:\n' 
         + '\033[0m')

clean_merged_campaign_dataframe.describe()

Table 2.3.4: Clean Merged Campaign DataFrame Description:



,cf_id,contact_id,goal,pledged,backers_count
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1585.743000,3751.434000,43983.100000,42748.055000,727.005000
std,926.944534,1431.743284,58962.425595,57415.481551,1137.723135
min,23.000000,1235.000000,100.000000,0.000000,0.000000
25%,758.750000,2502.750000,4200.000000,5911.250000,86.000000
50%,1571.500000,3706.500000,8300.000000,11950.000000,184.500000
75%,2402.250000,4994.250000,80625.000000,66294.250000,923.750000
max,3209.000000,6234.000000,199200.000000,199110.000000,7295.000000


### Clean Merged Campaign DataFrame Memory Usage

In [36]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'Table 2.3.5: Clean Merged Campaign DataFrame Memory Usage:\n' 
         + '\033[0m')

clean_merged_campaign_dataframe.memory_usage()

Table 2.3.5: Clean Merged Campaign DataFrame Memory Usage:



Index              132
cf_id             8000
contact_id        8000
company_name      8000
description       8000
goal              8000
pledged           8000
outcome           8000
backers_count     8000
country           8000
currency          8000
launch_date       8000
end_date          8000
category_id       8000
subcategory_id    8000
dtype: int64

## **2.4 Export Clean Merged Campaign DataFrame to CSV File**

In [37]:
clean_merged_campaign_dataframe.to_csv(CONSTANT_CAMPAIGN_FILE_PATH, index = False)

# <br> **3. Contacts DataFrame**

## **3.1 Create Contacts DataFrame**

### Initial Contacts DataFrame from Excel Workbook

In [38]:
excel_contacts_dtype_dictionary \
    = {'contact_id': 
            int,
       'name':
            str,
       'email':
            str}

contacts_dataframe \
    = pd.read_excel \
        (open(CONSTANT_CONTACTS_EXCEL_FILE_PATH, 'rb'),
         sheet_name = 'contact_info',
         header = 3,
         dtype = excel_contacts_dtype_dictionary)

### Display Initial Contacts DataFrame

In [39]:
pandas_process_functions \
    .return_formatted_table \
        (contacts_dataframe,
         'Table 3.1.1: Initial Contacts DataFrame')

contact_info
"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}"
"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}"
"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}"
"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}"
"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}"
"{""contact_id"": 5650, ""name"": ""Socorro Luna"", ""email"": ""socorro.luna@hotmail.com""}"
"{""contact_id"": 5889, ""name"": ""Carolina Murray"", ""email"": ""carolina.murray@knight.com""}"
"{""contact_id"": 4842, ""name"": ""Kayla Moon"", ""email"": ""kayla.moon@yahoo.de""}"
"{""contact_id"": 3280, ""name"": ""Ariadna Geisel"", ""email"": ""ariadna.geisel@rangel.com""}"
"{""contact_id"": 5468, ""name"": ""Danielle Ladeck"", ""email"": ""danielle.ladeck@scalfaro.net""}"


## **3.2 Extract Columns With Regular Expressions**

### Extract Contact ID And Add New Column

In [40]:
search_string = '(\d{4})'

contacts_dataframe['contact_id'] \
    = contacts_dataframe['contact_info'].str.extract(search_string)

log_subroutines \
    .log_write_object \
        (search_string \
         + '\n\n' 
         + str(contacts_dataframe['contact_info']))

### Display Updated Contacts DataFrame With Contact ID

In [41]:
pandas_process_functions \
    .return_formatted_table \
        (contacts_dataframe,
         'Table 3.2.1: Updated Contacts DataFrame with Contact ID') \
            .format({'contact_id': '{:}'})

contact_info,contact_id
"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}",4661
"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}",3765
"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}",4187
"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}",4941
"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}",2199
"{""contact_id"": 5650, ""name"": ""Socorro Luna"", ""email"": ""socorro.luna@hotmail.com""}",5650
"{""contact_id"": 5889, ""name"": ""Carolina Murray"", ""email"": ""carolina.murray@knight.com""}",5889
"{""contact_id"": 4842, ""name"": ""Kayla Moon"", ""email"": ""kayla.moon@yahoo.de""}",4842
"{""contact_id"": 3280, ""name"": ""Ariadna Geisel"", ""email"": ""ariadna.geisel@rangel.com""}",3280
"{""contact_id"": 5468, ""name"": ""Danielle Ladeck"", ""email"": ""danielle.ladeck@scalfaro.net""}",5468


### Contacts DataFrame Information

In [42]:
log_subroutines \
    .log_write_object \
        ('\033[1m' \
         + 'Table 3.2.2: Contacts DataFrame Information:\n' 
         + '\033[0m')

contacts_dataframe \
    .info \
        (verbose = True,
         memory_usage = True, 
         show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   contact_info  1000 non-null   object
 1   contact_id    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


### Convert Appropriate Column to `int64` Data Type

In [43]:
contacts_dataframe['contact_id'] \
    = contacts_dataframe['contact_id'].astype(np.int64)
    
log_subroutines \
    .log_write_object \
        (str(type(contacts_dataframe['contact_id'])) \
         + '\n\n' \
         + str(contacts_dataframe['contact_id']))

### Contacts DataFrame Information with Updated Column Data Type

In [44]:
log_subroutines \
    .log_write_object \
        ('\033[1m' \
         + 'Table 3.2.3: Contacts DataFrame Information ' \
         + 'with Updated Column Data Type:\n' 
         + '\033[0m')

contacts_dataframe \
    .info \
        (verbose = True,
         memory_usage = True, 
         show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   contact_info  1000 non-null   object
 1   contact_id    1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


### Extract Name And Add New Column

In [45]:
search_string = r'([^name"\s][a-zA-Z]+\s+[a-zA-Z]+)'

contacts_dataframe['name'] = contacts_dataframe['contact_info'].str.extract(search_string)

log_subroutines \
    .log_write_object \
        (search_string \
         + '\n\n' 
         + str(contacts_dataframe['contact_info']))

### Display Updated Contacts DataFrame With Name

In [46]:
pandas_process_functions \
    .return_formatted_table \
        (contacts_dataframe,
         'Table 3.2.4: Updated Contacts DataFrame With Name') \
            .format({'contact_id': '{:}'})

contact_info,contact_id,name
"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}",4661,Cecilia Velasco
"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}",3765,Mariana Ellis
"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}",4187,Sofie Woods
"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}",4941,Jeanette Iannotti
"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}",2199,Samuel Sorgatz
"{""contact_id"": 5650, ""name"": ""Socorro Luna"", ""email"": ""socorro.luna@hotmail.com""}",5650,Socorro Luna
"{""contact_id"": 5889, ""name"": ""Carolina Murray"", ""email"": ""carolina.murray@knight.com""}",5889,Carolina Murray
"{""contact_id"": 4842, ""name"": ""Kayla Moon"", ""email"": ""kayla.moon@yahoo.de""}",4842,Kayla Moon
"{""contact_id"": 3280, ""name"": ""Ariadna Geisel"", ""email"": ""ariadna.geisel@rangel.com""}",3280,Ariadna Geisel
"{""contact_id"": 5468, ""name"": ""Danielle Ladeck"", ""email"": ""danielle.ladeck@scalfaro.net""}",5468,Danielle Ladeck


### Extract E-mail And Add New Column

In [47]:
search_string = r'"(\S+@\S+)"}'

contacts_dataframe['email'] = contacts_dataframe['contact_info'].str.extract(search_string)

log_subroutines \
    .log_write_object \
        (search_string \
         + '\n\n' 
         + str(contacts_dataframe['contact_info']))

### Display Updated Contacts DataFrame With E-mail

In [48]:
pandas_process_functions \
    .return_formatted_table \
        (contacts_dataframe,
         'Table 3.2.5: Updated Contacts DataFrame With E-mail') \
            .format({'contact_id': '{:}'})

contact_info,contact_id,name,email
"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}",4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}",3765,Mariana Ellis,mariana.ellis@rossi.org
"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}",4187,Sofie Woods,sofie.woods@riviere.com
"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}",4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}",2199,Samuel Sorgatz,samuel.sorgatz@gmail.com
"{""contact_id"": 5650, ""name"": ""Socorro Luna"", ""email"": ""socorro.luna@hotmail.com""}",5650,Socorro Luna,socorro.luna@hotmail.com
"{""contact_id"": 5889, ""name"": ""Carolina Murray"", ""email"": ""carolina.murray@knight.com""}",5889,Carolina Murray,carolina.murray@knight.com
"{""contact_id"": 4842, ""name"": ""Kayla Moon"", ""email"": ""kayla.moon@yahoo.de""}",4842,Kayla Moon,kayla.moon@yahoo.de
"{""contact_id"": 3280, ""name"": ""Ariadna Geisel"", ""email"": ""ariadna.geisel@rangel.com""}",3280,Ariadna Geisel,ariadna.geisel@rangel.com
"{""contact_id"": 5468, ""name"": ""Danielle Ladeck"", ""email"": ""danielle.ladeck@scalfaro.net""}",5468,Danielle Ladeck,danielle.ladeck@scalfaro.net


## **3.3 Transform Contacts DataFrame**

### Transformed Contacts DataFrame

In [49]:
transformed_contacts_dataframe \
    = contacts_dataframe \
        [['contact_id', 
          'name', 
          'email']]

log_subroutines.log_write_object(transformed_contacts_dataframe)

### Display Transformed Contacts DataFrame

In [50]:
pandas_process_functions \
    .return_formatted_table \
        (transformed_contacts_dataframe,
         'Table 3.3.1: Transformed Contacts DataFrame') \
            .format({'contact_id': '{:}'})

contact_id,name,email
4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
3765,Mariana Ellis,mariana.ellis@rossi.org
4187,Sofie Woods,sofie.woods@riviere.com
4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
2199,Samuel Sorgatz,samuel.sorgatz@gmail.com
5650,Socorro Luna,socorro.luna@hotmail.com
5889,Carolina Murray,carolina.murray@knight.com
4842,Kayla Moon,kayla.moon@yahoo.de
3280,Ariadna Geisel,ariadna.geisel@rangel.com
5468,Danielle Ladeck,danielle.ladeck@scalfaro.net


### Extract First Name and Add New Column

In [51]:
search_string = r'(\w+)(?:\s\w+)'

transformed_contacts_dataframe['first_name'] \
    = transformed_contacts_dataframe['name'].str.extract(search_string)

log_subroutines \
    .log_write_object \
        (search_string \
         + '\n\n' 
         + str(transformed_contacts_dataframe['name']))

### Extract Last Name and Add New Column

In [52]:
search_string = r'(?:\w+\s)(\w+)'

transformed_contacts_dataframe['last_name'] \
    = transformed_contacts_dataframe['name'].str.extract(search_string)

log_subroutines \
    .log_write_object \
        (search_string \
         + '\n\n' 
         + str(transformed_contacts_dataframe['name']))

### Display Transformed Contacts DataFrame With First and Last Names

In [53]:
pandas_process_functions \
    .return_formatted_table \
        (transformed_contacts_dataframe,
         'Table 3.3.2: Transformed Contacts DataFrame With First and Last Names') \
            .format({'contact_id': '{:}'})

contact_id,name,email,first_name,last_name
4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr,Cecilia,Velasco
3765,Mariana Ellis,mariana.ellis@rossi.org,Mariana,Ellis
4187,Sofie Woods,sofie.woods@riviere.com,Sofie,Woods
4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com,Jeanette,Iannotti
2199,Samuel Sorgatz,samuel.sorgatz@gmail.com,Samuel,Sorgatz
5650,Socorro Luna,socorro.luna@hotmail.com,Socorro,Luna
5889,Carolina Murray,carolina.murray@knight.com,Carolina,Murray
4842,Kayla Moon,kayla.moon@yahoo.de,Kayla,Moon
3280,Ariadna Geisel,ariadna.geisel@rangel.com,Ariadna,Geisel
5468,Danielle Ladeck,danielle.ladeck@scalfaro.net,Danielle,Ladeck


## 3.4 Clean Contacts DataFrame

### Drop Name Column

In [54]:
clean_contacts_dataframe \
    = transformed_contacts_dataframe.drop('name', axis = 1)

log_subroutines.log_write_object(clean_contacts_dataframe)

### Reorder Columns

In [55]:
clean_contacts_dataframe \
    = transformed_contacts_dataframe \
        [['contact_id',
          'first_name', 
          'last_name', 
          'email']]

log_subroutines.log_write_object(clean_contacts_dataframe)

### Display Clean Contacts DataFrame

In [56]:
pandas_process_functions \
    .return_formatted_table \
        (clean_contacts_dataframe,
         'Table 3.4.1: Clean Contacts DataFrame') \
            .format({'contact_id': '{:}'})

contact_id,first_name,last_name,email
4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
3765,Mariana,Ellis,mariana.ellis@rossi.org
4187,Sofie,Woods,sofie.woods@riviere.com
4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com
5650,Socorro,Luna,socorro.luna@hotmail.com
5889,Carolina,Murray,carolina.murray@knight.com
4842,Kayla,Moon,kayla.moon@yahoo.de
3280,Ariadna,Geisel,ariadna.geisel@rangel.com
5468,Danielle,Ladeck,danielle.ladeck@scalfaro.net


### Clean Contacts DataFrame Information

In [57]:
log_subroutines \
    .print_and_log_text \
        ('\033[1m' \
         + 'Table 3.4.2: Clean Contacts DataFrame Information:\n'
         + '\033[0m')

clean_contacts_dataframe \
    .info \
        (verbose = True,
         memory_usage = True, 
         show_counts = True)

Table 3.4.2: Clean Contacts DataFrame Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


### Clean Contacts DataFrame Memory Usage

In [58]:
pandas_process_functions \
    .return_formatted_table \
        (clean_contacts_dataframe,
         'Table 3.4.3: Clean Contacts DataFrame Memory Usage')

contact_id,first_name,last_name,email
"4,661",Cecilia,Velasco,cecilia.velasco@rodrigues.fr
"3,765",Mariana,Ellis,mariana.ellis@rossi.org
"4,187",Sofie,Woods,sofie.woods@riviere.com
"4,941",Jeanette,Iannotti,jeanette.iannotti@yahoo.com
"2,199",Samuel,Sorgatz,samuel.sorgatz@gmail.com
"5,650",Socorro,Luna,socorro.luna@hotmail.com
"5,889",Carolina,Murray,carolina.murray@knight.com
"4,842",Kayla,Moon,kayla.moon@yahoo.de
"3,280",Ariadna,Geisel,ariadna.geisel@rangel.com
"5,468",Danielle,Ladeck,danielle.ladeck@scalfaro.net


## **3.5 Export Clean Contacts DataFrame to CSV File**

In [59]:
clean_contacts_dataframe.to_csv(CONSTANT_CONTACTS_CSV_FILE_PATH, index = False)

In [60]:
#log_subroutine \
#    .EndProgramExecution()